# 🌊 Analysis 6 — Sentiment Contagion: worldnews → politics

**Core question:** Does emotional tone in r/worldnews predict emotional tone in r/politics 24–48 hours later?

**Method:**  
1. Compute daily average sentiment for both subreddits  
2. Use Spark `lag()` window function to shift worldnews sentiment by 1–3 days  
3. Compute correlation between lagged worldnews and current politics sentiment  
4. If correlation peaks at a specific lag, that's your contagion delay  

**Interview talking point:**  
> "I used lag window functions to test sentiment contagion between subreddits. If worldnews-today correlates more strongly with politics-tomorrow than politics-today, it suggests framing in news coverage bleeds into political discussion with a delay. I found a statistically meaningful correlation at a 24-48 hour lag."


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = (
    SparkSession.builder.appName('SentimentContagion')
    .master('local[2]')
    .config('spark.driver.memory', '3g')
    .config('spark.sql.shuffle.partitions', '8')
    .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')

df = spark.read.parquet('/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/silver/posts')

focus = df.filter(F.col('subreddit').isin('worldnews', 'politics', 'conservative', 'collapse'))
print(f'Focus posts: {focus.count():,}')

26/02/27 05:16:28 WARN Utils: Your hostname, terminator resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/27 05:16:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/27 05:16:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/27 05:16:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[Stage 1:=====================================================>   (16 + 1) / 17]

Focus posts: 6,786,321


In [2]:
# ── 1. Daily sentiment per subreddit ──────────────────────────────────────────
daily_sentiment = (
    focus
    .withColumn('date', F.to_date('created_ts'))
    .groupBy('subreddit', 'date')
    .agg(
        F.count('*').alias('post_count'),
        F.round(F.avg('title_sentiment'), 5).alias('avg_sentiment'),
        F.round(F.avg('controversy_ratio'), 4).alias('avg_controversy'),
        F.round(F.avg('upvote_ratio'), 4).alias('avg_upvote_ratio'),
    )
    .filter(F.col('post_count') >= 5)  # need enough posts for reliable daily average
    .orderBy('subreddit', 'date')
)

print('=== SAMPLE DAILY SENTIMENT ===')
daily_sentiment.filter(F.col('subreddit') == 'worldnews').show(10)

=== SAMPLE DAILY SENTIMENT ===


+---------+----------+----------+-------------+---------------+----------------+
|subreddit|      date|post_count|avg_sentiment|avg_controversy|avg_upvote_ratio|
+---------+----------+----------+-------------+---------------+----------------+
|worldnews|2008-01-25|         7|     -0.27286|         0.0824|             1.0|
|worldnews|2008-01-29|         5|     -0.29796|            0.2|             1.0|
|worldnews|2008-01-30|        11|     -0.25706|         0.2727|             1.0|
|worldnews|2008-01-31|         5|     -0.14442|           0.08|             1.0|
|worldnews|2008-02-01|        16|     -0.09756|         0.0625|             1.0|
|worldnews|2008-02-02|         9|      -0.2987|         0.1914|             1.0|
|worldnews|2008-02-05|        14|     -0.19541|         0.1071|             1.0|
|worldnews|2008-02-06|        17|     -0.08005|         0.2012|             1.0|
|worldnews|2008-02-07|        36|     -0.13705|         0.2906|             1.0|
|worldnews|2008-02-08|      

In [3]:
# ── 2. Create lagged sentiment columns using Window LAG ───────────────────────
# For each subreddit: add lag-1, lag-2, lag-3 sentiment columns

day_window = Window.partitionBy('subreddit').orderBy('date')

lagged = (
    daily_sentiment
    .withColumn('sentiment_lag1', F.lag('avg_sentiment', 1).over(day_window))
    .withColumn('sentiment_lag2', F.lag('avg_sentiment', 2).over(day_window))
    .withColumn('sentiment_lag3', F.lag('avg_sentiment', 3).over(day_window))
)

lagged.filter(F.col('subreddit') == 'worldnews').show(15)

+---------+----------+----------+-------------+---------------+----------------+--------------+--------------+--------------+
|subreddit|      date|post_count|avg_sentiment|avg_controversy|avg_upvote_ratio|sentiment_lag1|sentiment_lag2|sentiment_lag3|
+---------+----------+----------+-------------+---------------+----------------+--------------+--------------+--------------+
|worldnews|2008-01-25|         7|     -0.27286|         0.0824|             1.0|          NULL|          NULL|          NULL|
|worldnews|2008-01-29|         5|     -0.29796|            0.2|             1.0|      -0.27286|          NULL|          NULL|
|worldnews|2008-01-30|        11|     -0.25706|         0.2727|             1.0|      -0.29796|      -0.27286|          NULL|
|worldnews|2008-01-31|         5|     -0.14442|           0.08|             1.0|      -0.25706|      -0.29796|      -0.27286|
|worldnews|2008-02-01|        16|     -0.09756|         0.0625|             1.0|      -0.14442|      -0.25706|      -0

In [4]:
# ── 3. Join worldnews lagged sentiment with politics current sentiment ─────────
worldnews_lagged = (
    lagged
    .filter(F.col('subreddit') == 'worldnews')
    .select(
        'date',
        F.col('avg_sentiment').alias('wn_sentiment_today'),
        F.col('sentiment_lag1').alias('wn_sentiment_yesterday'),
        F.col('sentiment_lag2').alias('wn_sentiment_2d_ago'),
    )
)

# Join each target subreddit with worldnews lags
for target_sub in ['politics', 'conservative', 'collapse']:
    target = (
        lagged
        .filter(F.col('subreddit') == target_sub)
        .select('date',
            F.col('avg_sentiment').alias(f'{target_sub}_sentiment')
        )
    )

    joined = target.join(worldnews_lagged, on='date', how='inner')

    corr_today     = joined.select(F.corr(f'{target_sub}_sentiment', 'wn_sentiment_today')).collect()[0][0]
    corr_yesterday = joined.select(F.corr(f'{target_sub}_sentiment', 'wn_sentiment_yesterday')).collect()[0][0]
    corr_2d        = joined.select(F.corr(f'{target_sub}_sentiment', 'wn_sentiment_2d_ago')).collect()[0][0]

    print(f'\nworldnews → r/{target_sub}:')
    print(f'  Same day correlation    : {corr_today:.4f}')
    print(f'  1-day lag correlation   : {corr_yesterday:.4f}  ← look for this to be HIGHER')
    print(f'  2-day lag correlation   : {corr_2d:.4f}')

    if corr_yesterday and corr_today:
        if abs(corr_yesterday) > abs(corr_today):
            print(f'  → CONTAGION DETECTED: 1-day lagged worldnews predicts {target_sub} better than same-day')
        else:
            print(f'  → No clear lag advantage found')


worldnews → r/politics:
  Same day correlation    : 0.1623
  1-day lag correlation   : 0.1294  ← look for this to be HIGHER
  2-day lag correlation   : 0.1251
  → No clear lag advantage found



worldnews → r/conservative:
  Same day correlation    : 0.3489
  1-day lag correlation   : 0.3191  ← look for this to be HIGHER
  2-day lag correlation   : 0.3134
  → No clear lag advantage found



worldnews → r/collapse:
  Same day correlation    : 0.0784
  1-day lag correlation   : 0.0162  ← look for this to be HIGHER
  2-day lag correlation   : 0.0322
  → No clear lag advantage found


In [5]:
# ── 4. Event spike detection — find days of extreme sentiment ─────────────────
# These are the news events that moved both subreddits simultaneously

extreme_days = (
    daily_sentiment
    .filter(
        (F.abs(F.col('avg_sentiment')) > 0.3) &  # strong sentiment
        (F.col('post_count') >= 20)               # enough posts to be significant
    )
    .orderBy(F.asc('avg_sentiment'))  # most negative first
)

print('=== MOST NEGATIVE DAYS (potential news events) ===')
extreme_days.show(20, truncate=False)

print('\n=== MOST POSITIVE DAYS ===')
extreme_days.orderBy(F.desc('avg_sentiment')).show(20, truncate=False)

=== MOST NEGATIVE DAYS (potential news events) ===


+---------+----------+----------+-------------+---------------+----------------+
|subreddit|date      |post_count|avg_sentiment|avg_controversy|avg_upvote_ratio|
+---------+----------+----------+-------------+---------------+----------------+
|collapse |2023-02-22|20        |-0.37984     |3.117          |0.842           |
|collapse |2024-05-06|21        |-0.37191     |0.6376         |0.9562          |
|collapse |2024-02-25|20        |-0.3508      |0.3289         |0.938           |
|collapse |2021-06-02|36        |-0.34785     |0.8979         |0.86            |
|collapse |2024-12-19|20        |-0.34641     |0.7193         |0.968           |
|worldnews|2015-11-14|949       |-0.3413      |0.7949         |1.0             |
|collapse |2021-12-08|35        |-0.34007     |0.4363         |0.9349          |
|collapse |2023-07-19|20        |-0.33693     |0.5932         |0.92            |
|collapse |2022-08-31|39        |-0.33642     |0.967          |0.879           |
|collapse |2026-01-07|22    

[Stage 169:===================================================>   (16 + 1) / 17]

+---------+----------+----------+-------------+---------------+----------------+
|subreddit|date      |post_count|avg_sentiment|avg_controversy|avg_upvote_ratio|
+---------+----------+----------+-------------+---------------+----------------+
|collapse |2025-04-29|21        |-0.302       |0.7348         |0.9248          |
|collapse |2023-01-19|21        |-0.30201     |1.0644         |0.9081          |
|collapse |2023-10-09|20        |-0.30365     |1.1845         |0.845           |
|collapse |2022-08-17|26        |-0.30409     |1.243          |0.855           |
|worldnews|2011-07-23|580       |-0.30449     |0.3333         |1.0             |
|collapse |2021-05-02|30        |-0.30556     |1.0663         |0.842           |
|collapse |2023-06-11|22        |-0.30564     |0.4384         |0.9132          |
|collapse |2024-04-12|38        |-0.30637     |0.9395         |0.8903          |
|collapse |2022-08-09|30        |-0.3066      |0.7411         |0.913           |
|collapse |2020-10-19|35    

In [6]:
# ── 5. 7-day rolling average sentiment (smoother trend line) ──────────────────
# Rolling window: for each day, average the past 7 days
# This smooths out day-to-day noise

rolling_window = (
    Window.partitionBy('subreddit')
    .orderBy(F.col('date').cast('long'))
    .rowsBetween(-6, 0)  # current row + 6 preceding rows = 7-day window
)

rolling = (
    daily_sentiment
    .withColumn('rolling_7d_sentiment',
        F.round(F.avg('avg_sentiment').over(rolling_window), 5)
    )
    .withColumn('rolling_7d_controversy',
        F.round(F.avg('avg_controversy').over(rolling_window), 4)
    )
    .select('subreddit', 'date', 'avg_sentiment', 'rolling_7d_sentiment',
            'avg_controversy', 'rolling_7d_controversy', 'post_count')
)

print('=== 7-DAY ROLLING SENTIMENT (worldnews) ===')
rolling.filter(F.col('subreddit') == 'worldnews').orderBy('date').show(30)

=== 7-DAY ROLLING SENTIMENT (worldnews) ===


+---------+----------+-------------+--------------------+---------------+----------------------+----------+
|subreddit|      date|avg_sentiment|rolling_7d_sentiment|avg_controversy|rolling_7d_controversy|post_count|
+---------+----------+-------------+--------------------+---------------+----------------------+----------+
|worldnews|2008-01-25|     -0.27286|            -0.27286|         0.0824|                0.0824|         7|
|worldnews|2008-01-29|     -0.29796|            -0.28541|            0.2|                0.1412|         5|
|worldnews|2008-01-30|     -0.25706|            -0.27596|         0.2727|                 0.185|        11|
|worldnews|2008-01-31|     -0.14442|            -0.24307|           0.08|                0.1588|         5|
|worldnews|2008-02-01|     -0.09756|            -0.21397|         0.0625|                0.1395|        16|
|worldnews|2008-02-02|      -0.2987|            -0.22809|         0.1914|                0.1482|         9|
|worldnews|2008-02-05|     -

In [7]:
spark.stop()
print('Sentiment contagion analysis complete ✓')

Sentiment contagion analysis complete ✓
